In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. 필요한 패키지 설치
!pip install fastapi pyngrok uvicorn fsspec==2025.3.2 transformers datasets accelerate nest_asyncio --quiet

In [ ]:
# 3. 주요 라이브러리 로딩
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
# 4. Math 문제 생성 모델 로딩
math_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Math/Problem"
math_problem_tokenizer = AutoTokenizer.from_pretrained(math_problem_model_path)
math_problem_model = AutoModelForCausalLM.from_pretrained(math_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 5. Math 정답 생성 모델 로딩
math_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Math/Answer"
math_answer_tokenizer = AutoTokenizer.from_pretrained(math_answer_model_path)
math_answer_model = AutoModelForCausalLM.from_pretrained(math_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 6. Logic 문제 생성 모델 로딩
logic_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Logic/Problem"
logic_problem_tokenizer = AutoTokenizer.from_pretrained(logic_problem_model_path)
logic_problem_model = AutoModelForCausalLM.from_pretrained(logic_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 7. Logic 정답 생성 모델 로딩
logic_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Logic/Answer"
logic_answer_tokenizer = AutoTokenizer.from_pretrained(logic_answer_model_path)
logic_answer_model = AutoModelForCausalLM.from_pretrained(logic_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 8. Knowledge 문제 생성 모델 로딩
knowledge_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Problem"
knowledge_problem_tokenizer = AutoTokenizer.from_pretrained(knowledge_problem_model_path)
knowledge_problem_model = AutoModelForCausalLM.from_pretrained(knowledge_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 9. Knowledge 정답 생성 모델 로딩
knowledge_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Answer"
knowledge_answer_tokenizer = AutoTokenizer.from_pretrained(knowledge_answer_model_path)
knowledge_answer_model = AutoModelForCausalLM.from_pretrained(knowledge_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 10. Open Problem 문제 생성 모델 로딩
open_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Open/Problem"
open_problem_tokenizer = AutoTokenizer.from_pretrained(open_problem_model_path)
open_problem_model = AutoModelForCausalLM.from_pretrained(open_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 11. 문제 생성 함수
def problem(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        temperature=0.8,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.convert_tokens_to_ids("<END>")
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("[문제생성결과]\n", generated_text)
    return generated_text.split("문제: ")[1].strip()

In [ ]:
# 12. 정답 생성 함수
def answer(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        temperature=0.8,
        top_p=0.8,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.convert_tokens_to_ids("<END>")
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("[정답생성결과]\n", generated_text)
    print()
    return generated_text.split("정답: ")[1].strip()

In [ ]:
# 13. FastAPI app 설정
app = FastAPI()

class ProblemRequest(BaseModel):
    topic: str
    level: str
    count: int

class ProblemResponse(BaseModel):
    response: list[str]

@app.post("/generate", response_model=ProblemResponse)
def generate_problems(req: ProblemRequest):
    global math_problem_model, math_problem_tokenizer, math_answer_model, math_answer_tokenizer
    global logic_problem_model, logic_problem_tokenizer, logic_answer_model, logic_answer_tokenizer
    global knowledge_problem_model, knowledge_problem_tokenizer, knowledge_answer_model, knowledge_answer_tokenizer
    global open_problem_model, open_problem_tokenizer

    topic = req.topic
    problem_list = []

    if topic == '수학':
      problem_model = math_problem_model
      problem_tokenizer = math_problem_tokenizer
      answer_model = math_answer_model
      answer_tokenizer = math_answer_tokenizer
    elif topic == '논리':
      problem_model = logic_problem_model
      problem_tokenizer = logic_problem_tokenizer
      answer_model = logic_answer_model
      answer_tokenizer = logic_answer_tokenizer
    elif topic == '상식':
      problem_model = knowledge_problem_model
      problem_tokenizer = knowledge_problem_tokenizer
      answer_model = knowledge_answer_model
      answer_tokenizer = knowledge_answer_tokenizer
    elif topic == '열린문제':
      problem_model = open_problem_model
      problem_tokenizer = open_problem_tokenizer
    else:
      return ProblemResponse(response=problem_list)

    for _ in range(req.count):
        # problem_prompt = f"난이도: {req.level}\n문제: "
        problem_prompt = f"문제: "
        problem_result = problem(problem_prompt, problem_model, problem_tokenizer)

        if topic == '열린문제':
          answer_result = '없음'
        else:
          answer_prompt = f"문제: {problem_result}\n정답: "
          answer_result = answer(answer_prompt, answer_model, answer_tokenizer)

        temp = f"문제: {problem_result}\n정답: {answer_result}"
        problem_list.append(temp)

    return ProblemResponse(response=problem_list)

In [ ]:
# 14. Ngrok 실행
ngrok.set_auth_token("***")
ngrok.kill()

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)